In [126]:
import numpy as np
import os

from PIL import Image

ASSUMTION_SIGNATURE = "a"
CUSTOM_UNSAT_HEAD = ""

In [195]:
n_assumptions = 10000
n_mucs = 10

density = 0.05
verbose = 0

mucs = []

while True:
    assignment = np.random.binomial(n=1, p=density, size=n_assumptions)
    candidate = {i + 1 for i, x in enumerate(assignment) if x == 1}
    if verbose:
        print(assignment, candidate)

    if any([candidate.issubset(muc) or muc.issubset(candidate) for muc in mucs]):
        if verbose:
            print("DENIED")
    else:
        if verbose:
            print("APPROVED")
        mucs.append(candidate)
        if len(mucs) >= n_mucs:
            break

In [196]:
print(len(mucs))
for muc in mucs:
    print(muc)

dirname = f"{n_assumptions}_assumptions"
if not os.path.isdir(dirname):
    os.mkdir(dirname)

img_width = 1280
img_height = int((n_mucs / n_assumptions) * img_width)
img = np.zeros(shape=(n_mucs, n_assumptions), dtype=np.uint8) + 255
for m, muc in enumerate(mucs):
    for a in muc:
        img[m, a - 1] = 0
Image.fromarray(img).resize((img_width, img_height), resample=Image.BOX).save(f"{dirname}/benchmark_{n_assumptions}assumptions_{n_mucs}mucs.jpeg")

with open(f"{dirname}/benchmark_{n_assumptions}assumptions_{n_mucs}mucs.txt", "w") as f:
    f.write("% ---[ AUTO GENERATED MUC BENCHMARK ]\n")
    f.write(f"% ---[ assumptions : {n_assumptions} ]\n")
    f.write(f"% ---[ mucs : {n_mucs} ]\n\n")

    f.write(f"num(1..{n_assumptions}).\n")
    f.write("{" + ASSUMTION_SIGNATURE + "(X): num(X)}.\n\n")

    for muc in mucs:
        f.write("{CUSTOM_UNSAT_HEAD}:- " + ", ".join([f"{ASSUMTION_SIGNATURE}({a})" for a in muc]) + ".\n")

10
{4106, 4108, 4113, 25, 4126, 2090, 2093, 8237, 6200, 4154, 6210, 8259, 8265, 6218, 6220, 4177, 6226, 6232, 90, 91, 93, 8297, 8311, 6281, 4241, 2199, 6296, 8352, 168, 6313, 8373, 4280, 8380, 8381, 4286, 8392, 8396, 2254, 208, 4304, 4306, 2260, 222, 6366, 2277, 235, 2283, 4333, 6388, 6406, 2312, 4360, 8456, 8462, 2326, 285, 2333, 2335, 8487, 2351, 8501, 8504, 6458, 8516, 2376, 330, 6478, 4433, 4436, 341, 2395, 2396, 6495, 2413, 2416, 2424, 4477, 383, 4484, 2438, 4490, 6542, 4495, 401, 6558, 2463, 4517, 2475, 4523, 8619, 4534, 455, 6600, 459, 8651, 2509, 8652, 4571, 480, 482, 8677, 486, 4583, 4585, 494, 6641, 6649, 4602, 509, 6656, 4618, 8715, 2580, 6691, 6700, 4655, 6703, 563, 6709, 8760, 4666, 6717, 6728, 8777, 4688, 593, 6745, 2653, 4705, 612, 8813, 8818, 8820, 4738, 8837, 2707, 8857, 668, 676, 678, 8875, 2737, 6834, 8881, 8882, 8884, 6844, 4797, 6845, 709, 2760, 2761, 2762, 2764, 4812, 8911, 8916, 6870, 4825, 8922, 737, 6889, 6896, 776, 6935, 6945, 8995, 9005, 2873, 6975, 6980, 699